In [ ]:
%pip install llama-index-readers-elasticsearch
%pip install llama-index-vector-stores-opensearch
%pip install llama-index-embeddings-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 29.4 MB/s eta 0:00:00


In [ ]:
from os import getenv
from llama_index.core import SimpleDirectoryReader
from llama_index.vector_stores.opensearch import (
    OpensearchVectorStore,
    OpensearchVectorClient,
)
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
import openai
from llama_index.core import PromptTemplate
from llama_index.llms.openai import OpenAI
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
endpoint = "https://search-document-index-v6c64wyx5etjcehscyfsfyv6tq.eu-north-1.es.amazonaws.com"

# index to demonstrate the VectorStore impl
idx = "document-index"

# OpensearchVectorClient stores text in this field by default
text_field = "content"
# OpensearchVectorClient stores embeddings in this field by default
embedding_field = "embedding"

# Provide the necessary credentials
auth = ('username', 'password')

client = OpensearchVectorClient(
    endpoint=endpoint,
    index=idx,
    dim=768,
    embedding_field=embedding_field,
    text_field=text_field,
    http_auth = auth,
)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Embedding Model Setting
embed_model = HuggingFaceEmbedding(
    model_name='mboth/distil-eng-quora-sentence'
)

In [ ]:
len(embed_model.get_text_embedding("Who was the director of a firm?"))

768

In [ ]:
!mkdir docs

mkdir: cannot create directory ‘docs’: File exists


In [ ]:
# load documents
documents = SimpleDirectoryReader('docs').load_data()

print(f"Length of the Documents: {len(documents)}")

Length of the Documents: 250


In [ ]:
splitter = SentenceSplitter(
    chunk_size=512,
    chunk_overlap=120,
)
nodes = splitter.get_nodes_from_documents(documents)

print(f"Length of Nodes: {len(nodes)}")

Length of Nodes: 294


In [ ]:
for i in range(len(nodes)-1):
  if nodes[i].text == '':
    del nodes[i]
print(len(nodes))

293


In [ ]:
nodes[1]

TextNode(id_='68e0af32-68b3-46c6-be06-4dfe03eb7613', embedding=None, metadata={'page_label': '2', 'file_name': 'Harry Potter and the Sorcerers Stone.pdf', 'file_path': '/content/docs/Harry Potter and the Sorcerers Stone.pdf', 'file_type': 'application/pdf', 'file_size': 1082762, 'creation_date': '2024-05-05', 'last_modified_date': '2024-05-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='3af61903-386b-4ec5-9326-9dffee9fe3ab', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '2', 'file_name': 'Harry Potter and the Sorcerers Stone.pdf', 'file_path': '/content/docs/Harry Potter and the Sorcerers Stone.pdf', 'file_type': 'application/pdf', 'file_size': 1082762, 'creation_date': '2024-05-05', '

In [ ]:
%%time

# initialize vector store
vector_store = OpensearchVectorStore(client)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
# initialize an index using our sample data and the client we just created
index = VectorStoreIndex(
    nodes=nodes,
    storage_context=storage_context,
    insert_batch_size=10,
    embed_model = embed_model,
    show_progress=True,
)

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 4min 9s, sys: 34.6 s, total: 4min 43s
Wall time: 5min 24s


# DEFAULT APPROACH

In [ ]:
openai.api_key = "Open API Key"

In [ ]:
custom_qa_prompt = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query.\n\n"
    "Note: If you don't get any context information, then just appologize that you don't "
    "have information about it rather than generating answer on your own.\n"
    "Furthermore, you are only required to communicate with user in ENGLISH LANGUAGE."
    "Query: {query_str}\n"
    "Answer: "
)

custom_qa_prompt = PromptTemplate(custom_qa_prompt)

# Making Query Engine

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-0613")

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=20,
)


response_synthesizer = get_response_synthesizer(text_qa_template=custom_qa_prompt,)

default_query_engine = RetrieverQueryEngine.from_args(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    llm=llm,
)

In [ ]:
response = default_query_engine.query("Who is Dursley?")
print(response)

The Dursleys are the family that Harry Potter stays with at the beginning of the story. They consist of Mr. and Mrs. Dursley, as well as their son Dudley.


In [ ]:
from llama_index.core import PromptTemplate

custom_prompt = PromptTemplate(
    """\
Given a conversation (between Human and Assistant) and a follow up message from Human, \
rewrite the message to be a standalone question that captures all relevant context \
from the conversation.

<Chat History>
{chat_history}

<Follow Up Message>
{question}

<Standalone question>
"""
)

# Making Chat Engine on the top of Query Engine

In [ ]:
from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

In [ ]:
from llama_index.core.chat_engine import CondenseQuestionChatEngine

default_chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=default_query_engine,
    condense_question_prompt=custom_prompt,
    memory=memory,
)

# Full conversation with chatbot

In [ ]:
chatbot = default_chat_engine.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.

Human: Who was the Dursley?
Assistant: The Dursleys are a family consisting of Mr. Dursley, Mrs. Dursley, and their son Dudley. They are relatives of Harry Potter's mother, but they have kept their connection to the magical world a secret.

Human: Who is the Albus Dumbledore?
Assistant: Albus Dumbledore is a prominent figure in the magical world of Harry Potter, serving as the headmaster of Hogwarts School of Witchcraft and Wizardry. Known for his wisdom and power, Dumbledore is revered as one of the most skilled wizards of his time. He has a significant connection to Harry Potter, offering guidance and protection to the young wizard on his adventures.

Human: exit
